In [17]:

import psycopg2
from psycopg2.extras import execute_values


In [4]:
# Parámetros de conexión – AJUSTA esto a tu configuración local
conn = psycopg2.connect(
    host="localhost",
    database="PROYECTO_FINAL_AMAZON",
    user="postgres",
    password="quique"
)
cursor = conn.cursor()
print("Conexión establecida correctamente.")


Conexión establecida correctamente.


In [5]:
# Crear tabla categoria
cursor.execute("""
CREATE TABLE IF NOT EXISTS categoria (
    id SERIAL PRIMARY KEY,
    nombre TEXT NOT NULL UNIQUE
);
""")

# Crear tabla subcategoria
cursor.execute("""
CREATE TABLE IF NOT EXISTS subcategoria (
    id SERIAL PRIMARY KEY,
    nombre TEXT NOT NULL,
    categoria_id INTEGER REFERENCES categoria(id)
);
""")

# Crear tabla producto
cursor.execute("""
CREATE TABLE IF NOT EXISTS producto (
    id SERIAL PRIMARY KEY,
    nombre TEXT NOT NULL,
    descripcion TEXT,
    precio NUMERIC NOT NULL,
    precio_original NUMERIC,
    porcentaje_descuento NUMERIC,
    porcentaje_descuento_2 NUMERIC,
    no_ratings INTEGER,
    asin TEXT,
    image TEXT,
    link TEXT,
    subcategoria_id INTEGER REFERENCES subcategoria(id)
);
""")

conn.commit()
print("Tablas creadas correctamente.")


Tablas creadas correctamente.


In [8]:
import pandas as pd

# Ruta al archivo CSV limpio 
df = pd.read_csv("c:/Users/User/Desktop/hackio/proyectos/Proyecto_Final/Proyecto_final/data/data_procesed/amazon_data_clean_eda.csv")

# Confirmamos que se cargó correctamente
print("DataFrame cargado con shape:", df.shape)
df.head()


DataFrame cargado con shape: (738060, 12)


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,discount_percent,asin,discount_percentage
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,2255,32999.0,58990.0,44.0,B0BRKXTSBT,44.060010
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,2948,46490.0,75990.0,39.0,B0BQ3MXML8,38.820897
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,1206,34490.0,61990.0,44.0,B0BPYN9JGF,44.361994
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,37990.0,68990.0,45.0,B0BQ3MJ1TG,44.934048
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,34490.0,67790.0,49.0,B0B67RLLJC,49.122289


In [9]:
categorias = df['main_category'].dropna().drop_duplicates()

for cat in categorias:
    cursor.execute("""
        INSERT INTO categoria (nombre)
        VALUES (%s)
        ON CONFLICT (nombre) DO NOTHING;
    """, (cat,))
conn.commit()
print("Categorías insertadas.")


Categorías insertadas.


In [13]:
# Creamos diccionario de categorías
cursor.execute("SELECT id, nombre FROM categoria")
categoria_map = {nombre: id for id, nombre in cursor.fetchall()}

In [14]:
# Insertar subcategorías
subcategorias = df[['main_category', 'sub_category']].drop_duplicates()

for _, row in subcategorias.iterrows():
    nombre_sub = row['sub_category']
    nombre_cat = row['main_category']
    categoria_id = categoria_map.get(nombre_cat)

    if categoria_id:
        cursor.execute("""
            INSERT INTO subcategoria (nombre, categoria_id)
            VALUES (%s, %s)
            ON CONFLICT DO NOTHING;
        """, (nombre_sub, categoria_id))
conn.commit()
print("Subcategorías insertadas.")


Subcategorías insertadas.


In [15]:
# Cremos diccionario para mapear subcategorías a sus IDs
cursor.execute("SELECT id, nombre FROM subcategoria")
subcategoria_map = {nombre: id for id, nombre in cursor.fetchall()}

In [18]:
# Insertamos productos

productos_data = []

for _, row in df.iterrows():
    subcat_id = subcategoria_map.get(row['sub_category'])
    if subcat_id:
        productos_data.append((
            row['name'],
            row['ratings'],
            row['discount_price'],
            row['actual_price'],
            row.get('discount_percent'),
            row.get('discount_percentage'),
            row['no_of_ratings'],
            row['asin'],
            row['image'],
            row['link'],
            subcat_id
        ))

# Ahora insertamos todos los productos en lote
query = """
    INSERT INTO producto (
        nombre, descripcion, precio, precio_original,
        porcentaje_descuento, porcentaje_descuento_2,
        no_ratings, asin, image, link, subcategoria_id
    ) VALUES %s
"""
execute_values(cursor, query, productos_data)
conn.commit()
print("Inserción masiva completada.")



Inserción masiva completada.


In [19]:
# Cerramos la conexión
cursor.close()
conn.close()
print("Conexión cerrada.")

Conexión cerrada.
